In [2]:
import os
print(f"Default directory:{os.getcwd()}")
os.chdir("..")
print(f"New directory:{os.getcwd()}")

Default directory:C:\Users\danie\OneDrive\Escritorio\Sabadell\1. Python Projects\linkedin_scraper\4. Analisis
New directory:C:\Users\danie\OneDrive\Escritorio\Sabadell\1. Python Projects\linkedin_scraper


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import plotly.express as px
from datetime import date

In [4]:
#================================= Load Data ====================================
con = sqlite3.connect(r".\2. Database\linkedin_jobpostings.db")
df = pd.read_sql_query("SELECT * FROM JOBPOSTINGS", con)
con.close()

#### Fuzzy String Matching

Fuzzy String Matching, also known as *Approximate String Matching*, is the process of finding strings that approximately match a pattern. The process has various applications such as spell-checking, DNA analysis and detection, spam detection, plagiarism detection e.t.c

- Source 1: https://towardsdatascience.com/surprisingly-effective-way-to-name-matching-in-python-1a67328e670e
- Source 2: https://towardsdatascience.com/fuzzy-string-matching-in-python-68f240d910fe

**Example**

In [20]:
df.formattedLocation.unique()

array(['Barcelona, Catalonia, Spain',
       'San Sebastián, Basque Country, Spain',
       'Madrid, Community of Madrid, Spain', 'Málaga, Andalusia, Spain',
       'Greater Madrid Metropolitan Area', 'Spain',
       'Community of Madrid, Spain', 'Valencian Community, Spain',
       'Sant Cugat del Vallès, Catalonia, Spain',
       'Greater Barcelona Metropolitan Area',
       'Tarragona, Catalonia, Spain',
       'Valencia, Valencian Community, Spain',
       'El Prat de Llobregat, Catalonia, Spain',
       'Palau-solità i Plegamans, Catalonia, Spain',
       'Esplugues de Llobregat, Catalonia, Spain',
       'Las Rozas de Madrid, Community of Madrid, Spain',
       'Seville, Andalusia, Spain',
       'Majadahonda, Community of Madrid, Spain', 'Canary Islands, Spain',
       'Alicante, Valencian Community, Spain',
       'Santander, Cantabria, Spain',
       'Greater Palma de Mallorca Metropolitan Area',
       'Bilbao-Bilbo, Basque Country, Spain',
       'Greater Málaga Metropolitan

In [69]:
ccaa=[
        "Andalusia",
        "Aragon",
        "Asturias",
        "Balearic Islands",
        "Basque Country",
        "Canary Islands",
        "Cantabria",
        "Castile and León",
        "Castilla–La Mancha",
        "Catalonia",
        "Community of Madrid",
        "Extremadura",
        "Galicia",
        "La Rioja",
        "Navarre",
        "Region of Murcia",
        "Valencian Community",

]

####  Fuzzywuzzy library

Fuzzywuzzy is a python library that uses **Levenshtein Distance to calculate the differences between sequences and patterns** that was developed and also open-sourced by SeatGeek, a service that finds event tickets from all over the internet and showcase them on one platform. The big problem they were facing was the labeling of the same events as stated on their blog. This is the same as the example I gave at the beginning of the post where an entity such as a person’s name can be labelled differently on different sources.

In [22]:
from fuzzywuzzy import fuzz, process

In [73]:
#Defining a function to return the match and similarity score of the fuzz.ratio() scorer. The function will take in a term(name), list of terms(list_names), and a minimum similarity score(min_score) to return the match. 
def match_names(name, list_names, min_score=0):
    max_score = -1
    max_name = ''
    for x in list_names:
        score = fuzz.partial_ratio(name, x)
        if (score > min_score) & (score > max_score):
            max_name = x
            max_score = score
    return (max_name, max_score)

In [78]:
#For loop to create a list of tuples with the first value being the name from the second dataframe (name to replace) and the second value from the first dataframe (string replacing the name value). Then, casting the list of tuples as a dictionary. 
names = []
for x in df.formattedLocation:
    match = match_names(x, ccaa, 90)
    if match[1] >= 75:
        name = ('(' + str(x), str(match[0]) + ')')
        names.append(name)
name_dict = dict(names)
name_dict

{'(Barcelona, Catalonia, Spain': 'Catalonia)',
 '(San Sebastián, Basque Country, Spain': 'Basque Country)',
 '(Madrid, Community of Madrid, Spain': 'Community of Madrid)',
 '(Málaga, Andalusia, Spain': 'Andalusia)',
 '(Community of Madrid, Spain': 'Community of Madrid)',
 '(Valencian Community, Spain': 'Valencian Community)',
 '(Sant Cugat del Vallès, Catalonia, Spain': 'Catalonia)',
 '(Tarragona, Catalonia, Spain': 'Catalonia)',
 '(Valencia, Valencian Community, Spain': 'Valencian Community)',
 '(El Prat de Llobregat, Catalonia, Spain': 'Catalonia)',
 '(Palau-solità i Plegamans, Catalonia, Spain': 'Catalonia)',
 '(Esplugues de Llobregat, Catalonia, Spain': 'Catalonia)',
 '(Las Rozas de Madrid, Community of Madrid, Spain': 'Community of Madrid)',
 '(Seville, Andalusia, Spain': 'Andalusia)',
 '(Majadahonda, Community of Madrid, Spain': 'Community of Madrid)',
 '(Canary Islands, Spain': 'Canary Islands)',
 '(Alicante, Valencian Community, Spain': 'Valencian Community)',
 '(Santander, Can

In [66]:
process.extractBests('Madrid ',
                     df.formattedLocation.unique(),
                     scorer=fuzz.partial_ratio,
                     score_cutoff=90, limit=None)

[('Madrid, Community of Madrid, Spain', 100),
 ('Greater Madrid Metropolitan Area', 100),
 ('Community of Madrid, Spain', 100),
 ('Las Rozas de Madrid, Community of Madrid, Spain', 100),
 ('Majadahonda, Community of Madrid, Spain', 100),
 ('San Fernando de Henares, Community of Madrid, Spain', 100),
 ('Tres Cantos, Community of Madrid, Spain', 100),
 ('Boadilla del Monte, Community of Madrid, Spain', 100),
 ('Getafe, Community of Madrid, Spain', 100),
 ('Pozuelo de Alarcón, Community of Madrid, Spain', 100),
 ('Pinto, Community of Madrid, Spain', 100)]

In [79]:
process.extractBests('Barcelona',
                     df.formattedLocation.unique(),
                     scorer=fuzz.partial_ratio,
                     score_cutoff=90, limit=None)

[('Barcelona, Catalonia, Spain', 100),
 ('Greater Barcelona Metropolitan Area', 100)]